In [1]:
from imutils.video import VideoStream
import face_recognition
import pickle
import cv2
import imutils
import time
import imageio

In [2]:
data = pickle.loads(open("output/encodings.pickle", "rb").read())

In [3]:
reader = imageio.get_reader('Dataset/cgc.mp4')
fps = reader.get_meta_data()['fps']
writer = imageio.get_writer('output/cgc.mp4', fps = fps)
for i, frame in enumerate(reader):
    
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    rgb = imutils.resize(frame, width=750)
    r = frame.shape[1] / float(rgb.shape[1])
    # detect the (x, y)-coordinates of the bounding boxes
    # corresponding to each face in the input frame, then compute
    # the facial embeddings for each face
    boxes = face_recognition.face_locations(rgb,
                                            model="cnn")
    encodings = face_recognition.face_encodings(rgb, boxes)
    names = []
    # Recognition step    
    for encoding in encodings:
    
        # comparing the extract images with the embeddings images from dataset
        matches = face_recognition.compare_faces(data["encodings"], encoding)
        name = "Unknown"
        if True in matches:
            # find the indexes of all matched faces then initialize a
            # dictionary to count the total number of times each face
            # was matched
            matchedIdxs = [i for (i, b) in enumerate(matches) if b]
            counts = {}
            # loop over the matched indexes and maintain a count for
            # each recognized face face
            for i in matchedIdxs:
                name = data["names"][i]
                counts[name] = counts.get(name, 0) + 1
            # determine the recognized face with the largest number of
            # votes (note: in the event of an unlikely tie Python will
            # select first entry in the dictionary)
            name = max(counts, key=counts.get)
        names.append(name)


    # update the list of names
    
    for ((top, right, bottom, left), name) in zip(boxes, names):
        # rescale the face coordinates
        top = int(top * r)
        right = int(right * r)
        bottom = int(bottom * r)
        left = int(left * r)
        # draw the predicted face name on the image
        cv2.rectangle(frame, (left, top), (right, bottom),
                      (0, 255, 0), 2)
        y = top - 15 if top - 15 > 15 else top + 15
        cv2.putText(frame, name, (left, y), cv2.FONT_HERSHEY_SIMPLEX,
                    0.75, (0, 255, 0), 2)
    # if the video writer is None *AND* we are supposed to write
    # the output video to disk initialize the writer

    
    writer.append_data(frame)
writer.close()